In [1]:
%load_ext autoreload
%autoreload 2

import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg, add the modules
import operator # sorting
from math import *
import matplotlib.pyplot as plt

sys.path.append('../')

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

import prep
from warp import *

In [2]:
trace_file = 'trace.csv'
metrics_file = 'metrics.csv'

# read trace and metrics into dataframe, from generated trace file
df_kernel_trace, df_metrics = prep.Prep_trace_metrics(trace_file, metrics_file)

In [3]:
unique_kerns = []
for index, kenname in enumerate(df_kernel_trace.Name.unique()):
    if index > 0 and 'memcpy' not in kenname:
        head = kenname.split('(')[0]
        if '<' in head:
            head = head.split('<')[0]
        #print head
    
        if 'void' in head:
            head = head.split(' ')
            #print head[1]
            head = head[1]
            
        if head not in unique_kerns:
            unique_kerns.append(head)
            print head

FWD_init_alpha
dot_kernel
reduce_1Block_kernel
FWD_scaling
gemv2N_kernel_val
FWD_calc_alpha
FWD_sum_ll
BK_update_beta
gemv2T_kernel_val
BK_scaling
EM_betaB_alphabeta
EM_alphabeta_update_gamma
EM_A_mul_alphabetaB
EM_update_xisum
EM_alphabeta
EM_expect_A
EM_transpose
EM_gammastatesum
EM_gammaobs
EM_expectmu
EM_expectsigma_dev
EM_update_expectsigma


In [4]:
target_kern_name = 'EM_gammaobs'

sass_result = target_kern_name + '.sm_52.sass.result'
kernel_stat_out ='kernelStat_' + target_kern_name + '.csv'

out_columns = prep.init_df_columns(df_metrics)
df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

warp_inst = WarpInst()
warp_inst.run(df_kern, sass_result)

df_kern.insert(1, 'm2c_ratio', warp_inst.mem_clks / warp_inst.cmp_clks)
df_kern.to_csv(kernel_stat_out, index=False, encoding='utf-8')
df_kern

LDG (global load) clocks (per warp) 		 = 650.0
STG (global store) clocks (per warp) 		 = 19.0
LDS (shared memory load) clocks (per warp) 	 = 0.0
STS (shared memory store) clocks (per warp) 	 = 0.0
=> Memory inst. (per warp) 			 = 669.0 (clocks)

Integer inst. (per warp) 		 = 240.0 (clocks)
FP32 inst. (per warp) 			 = 15.0 (clocks)
FP64 inst. (per warp) 			 = 0.0 (clocks)
Compute load store inst. (per warp) 	 = 45.0 (clocks)
=> Compute inst. (per warp) 		 = 300.0 (clocks)

Memory Intensive : mem / cmp = 2.23


,kern_name,m2c_ratio,duration_ms,gridx,gridy,gridz,blkx,blky,blkz,reg_per_thread,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,em_gammaobs,2.23,0.0016,4.0,4.0,1.0,16.0,16.0,1.0,8.0,...,Low (1),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),0.623941,0.051838%,0.000000%


In [5]:
target_kern_name = 'EM_expectmu'

sass_result = target_kern_name + '.sm_52.sass.result'
kernel_stat_out ='kernelStat_' + target_kern_name + '.csv'

out_columns = prep.init_df_columns(df_metrics)
df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

warp_inst = WarpInst()
warp_inst.run(df_kern, sass_result)

df_kern.insert(1, 'm2c_ratio', warp_inst.mem_clks / warp_inst.cmp_clks)
df_kern.to_csv(kernel_stat_out, index=False, encoding='utf-8')
df_kern

LDG (global load) clocks (per warp) 		 = 650.0
STG (global store) clocks (per warp) 		 = 19.0
LDS (shared memory load) clocks (per warp) 	 = 26.0
STS (shared memory store) clocks (per warp) 	 = 19.0
=> Memory inst. (per warp) 			 = 714.0 (clocks)

Integer inst. (per warp) 		 = 600.0 (clocks)
FP32 inst. (per warp) 			 = 75.0 (clocks)
FP64 inst. (per warp) 			 = 0.0 (clocks)
Compute load store inst. (per warp) 	 = 105.0 (clocks)
=> Compute inst. (per warp) 		 = 780.0 (clocks)

Compute Intensive : cmp / mem = 1.09243697479


,kern_name,m2c_ratio,duration_ms,gridx,gridy,gridz,blkx,blky,blkz,reg_per_thread,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,em_expectmu,0.915385,0.002112,1.0,4.0,1.0,16.0,16.0,1.0,18.0,...,Low (1),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),0.307958,0.051981%,0.000000%


In [6]:
target_kern_name = 'EM_expectsigma_dev'

sass_result = target_kern_name + '.sm_52.sass.result'
kernel_stat_out ='kernelStat_' + target_kern_name + '.csv'

out_columns = prep.init_df_columns(df_metrics)
df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

warp_inst = WarpInst()
warp_inst.run(df_kern, sass_result)

df_kern.insert(1, 'm2c_ratio', warp_inst.mem_clks / warp_inst.cmp_clks)
df_kern.to_csv(kernel_stat_out, index=False, encoding='utf-8')
df_kern

LDG (global load) clocks (per warp) 		 = 2600.0
STG (global store) clocks (per warp) 		 = 19.0
LDS (shared memory load) clocks (per warp) 	 = 104.0
STS (shared memory store) clocks (per warp) 	 = 19.0
=> Memory inst. (per warp) 			 = 2742.0 (clocks)

Integer inst. (per warp) 		 = 1440.0 (clocks)
FP32 inst. (per warp) 			 = 1125.0 (clocks)
FP64 inst. (per warp) 			 = 0.0 (clocks)
Compute load store inst. (per warp) 	 = 2460.0 (clocks)
=> Compute inst. (per warp) 		 = 5025.0 (clocks)

Compute Intensive : cmp / mem = 1.83260393873


,kern_name,m2c_ratio,duration_ms,gridx,gridy,gridz,blkx,blky,blkz,reg_per_thread,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,em_expectsigma_dev,0.545672,0.006208,8.0,8.0,1.0,8.0,8.0,1.0,25.0,...,Low (2),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),1.349711,3.644459%,0.000000%


In [7]:
target_kern_name = 'EM_update_expectsigma'

sass_result = target_kern_name + '.sm_52.sass.result'
kernel_stat_out ='kernelStat_' + target_kern_name + '.csv'

out_columns = prep.init_df_columns(df_metrics)
df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

warp_inst = WarpInst()
warp_inst.run(df_kern, sass_result)

df_kern.insert(1, 'm2c_ratio', warp_inst.mem_clks / warp_inst.cmp_clks)
df_kern.to_csv(kernel_stat_out, index=False, encoding='utf-8')
df_kern

LDG (global load) clocks (per warp) 		 = 650.0
STG (global store) clocks (per warp) 		 = 19.0
LDS (shared memory load) clocks (per warp) 	 = 0.0
STS (shared memory store) clocks (per warp) 	 = 0.0
=> Memory inst. (per warp) 			 = 669.0 (clocks)

Integer inst. (per warp) 		 = 255.0 (clocks)
FP32 inst. (per warp) 			 = 15.0 (clocks)
FP64 inst. (per warp) 			 = 0.0 (clocks)
Compute load store inst. (per warp) 	 = 30.0 (clocks)
=> Compute inst. (per warp) 		 = 300.0 (clocks)

Memory Intensive : mem / cmp = 2.23


,kern_name,m2c_ratio,duration_ms,gridx,gridy,gridz,blkx,blky,blkz,reg_per_thread,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,em_update_expectsigma,2.23,0.001472,4.0,4.0,1.0,16.0,16.0,1.0,8.0,...,Low (1),0.000000B/s,0,Low (1),0.000000B/s,Idle (0),Low (1),0.603971,0.000808%,0.000000%
